# Blue Team KubeHound Workflow

A step by step example workflow of how to use KubeHound for an incident response scenario.

## Initial Setup

Connect to the kubegraph server by running the cell below

In [ ]:
%%graph_notebook_config
{
  "host": "host.docker.internal",
  "port": 8182,
  "ssl": false,
  "gremlin": {
    "traversal_source": "g",
    "username": "",
    "password": "",
    "message_serializer": "graphsonv3"
  }
}

Now set the appearance customizations for the notebook. You can see a guide on possible options [here](https://github.com/aws/graph-notebook/blob/623d43827f798c33125219e8f45ad1b6e5b29513/src/graph_notebook/notebooks/01-Neptune-Database/02-Visualization/Grouping-and-Appearance-Customization-Gremlin.ipynb#L680)

In [ ]:
%%graph_notebook_vis_options
{
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    }
  }
}

## Workflow

### Compromised Credentials

Let us consider a scenario where a user's credentials have been compromised. We can use KubeHound to identify the resources that the user has access to, whether any lead to critical assets and what attacks might have been leveraged.

First let's see whether there are any critical paths accessible. Because Kubernetes delegates the management of users' group memberships to third party components (e.g identity providers), we need to check paths from both the user and any groups they are a member of.

In [ ]:
%%gremlin
kh.identities().
    or(
        has("type", "Group").has("name", within("dept-sales", "k8s-users")),
        has("type", "User").has("name", "bits.barkley@datadoghq.com")).
    hasCriticalPath().
    values("name")

Now let's examine the possible attack paths that could be taken by an attacker who has access to the compromised credentials.

In [ ]:

%%gremlin -d class -g critical -le 50 -p inv,oute
kh.users("bits.barkley@datadoghq.com").
    criticalPaths().
    by(elementMap()).
    limit(100)


Skip to the [next section](#advanced-workflows) to for more in-depth workflows to surface potential detection sources and eliminate attacks to narrow down the scope.

### Compromised Container

Consider the scenario where a container has been compromised via a malicious dependency found within a known set of images. We can use KubeHound to identify the resources that the container has access to, whether any lead to critical assets and what attacks might have been leveraged.

First let's see whether there are any critical paths accessible

In [ ]:
%%gremlin
kh.containers().
    or(
        has("image", TextP.containing("nginx")),
        has("image", TextP.containing("elasticsearch"))).
    hasCriticalPath().
    values("name").
    dedup()

Now let's examine the possible attack paths that could be taken by the attacker.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute
kh.containers().
    or(
        has("image", TextP.containing("nginx")),
        has("image", TextP.containing("elasticsearch"))).
    criticalPaths().
    by(elementMap()).
    limit(100)

## Advanced Workflows

In a real-world deployment the above queries may throw up too many results to be actionable. In such cases we can use KubeHound to narrow down the scope of the investigation.

For example in the compromised container case we may wish to understand the easiest attack path that the attacker could have taken and focus our detections efforts there. Let's first look for any potential container escapes from the compromised container. These provide easy privilege escalation for an attacker, but could provide detection opportunities for us. 

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute
kh.containers().
    or(
        has("image", TextP.containing("nginx")),
        has("image", TextP.containing("elasticsearch"))).
    attacks().
    by(elementMap()).
    limit(100)

find the shortest attack paths from out compromised containers.